In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
import cv2
import math
import json
import itertools as it

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim

from kaggle_camera_model_id_lib.utils import PechkaBot, ImageList, NpzFolder, NCrops, TifFolder, MultiDataset
from kaggle_camera_model_id_lib.models import VggHead, StyleVggHead, IEEEfcn, ResNetFC, ResNetX, FatNet1
from kaggle_camera_model_id_lib.models import InceptionResNetV2fc, InceptionResNetV2fcSmall
from kaggle_camera_model_id_lib.utils import jpg_compress, equalize_v_hist, hsv_convert
from kaggle_camera_model_id_lib.utils import scale_crop_pad, gamma_correction
from kaggle_camera_model_id_lib.utils import patch_quality_dich, n_random_crops, n_pseudorandom_crops
from kaggle_camera_model_id_lib.models import DANet, ResNetFeatureExtractor, AvgFcClassifier, FCDiscriminator
from kaggle_camera_model_id_lib.models import AvgClassifier, InceptionResNetV2, ResNetDense, ResNetDenseFC

import imgaug as ia
from imgaug import augmenters as iaa

import pretrainedmodels as pm

In [2]:
crop_center_size = 1520
crop_size = 256
random_crop = transforms.RandomCrop(crop_size)
center_crop = transforms.CenterCrop(crop_center_size)
rvf = transforms.RandomVerticalFlip()
rhf = transforms.RandomHorizontalFlip()
random_flip = lambda img: rvf(rhf(img))

n_crops_search_train = 36
n_crops_train = 9

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

In [3]:
train_list_path = '/home/mephistopheies/storage2/data/camera-model-id/train.tsv'
val_path = '/home/mephistopheies/storage2/data/camera-model-id/val/'
test_path = '/home/mephistopheies/storage2/data/camera-model-id/raw/test/'

In [2]:
X_var = Variable(torch.from_numpy(np.random.uniform(-1, 1, size=(5, 3, 256, 256)).astype(np.float32)))

In [2]:
x = [1.2, 1.4, 0.5]

In [4]:
# n_classes = 10

# model = InceptionResNetV2(num_classes=n_classes)

In [3]:
# model(X_var)

In [2]:
# model.features(X_var).shape

In [3]:
model = pm.InceptionResNetV2(num_classes=10)

In [5]:
model.features(X_var).shape

torch.Size([5, 1536, 5, 5])

In [3]:
model = ResNetDenseFC(models.resnet.BasicBlock, [3, 4, 6, 3], num_classes=10, load_resnet=None, 
                      zero_first_center=True)
model = model.train()

In [4]:
model = model.cuda()

In [5]:
criterion = nn.CrossEntropyLoss().cuda()

In [6]:
log_p = model(X_var.cuda())
Y_var = Variable(torch.LongTensor([0, 1, 2, 3, 4]).cuda())

In [7]:
loss = criterion(log_p, Y_var)

In [8]:
optimizer = optim.SGD(
    model.parameters(), 
    lr=0.1, 
    momentum=0.9, 
    dampening=0, 
    weight_decay=0.0,
    nesterov=True)

optimizer.zero_grad()
loss.backward()

In [18]:
optimizer.zero_grad()

In [19]:
p.grad

Variable containing:
(0 ,0 ,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0

(0 ,1 ,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0

(0 ,2 ,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ⋮ 

(1 ,0 ,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0

(1 ,1 ,.,.) = 
   0   0   0  ...    0  